## Deploy the Saved Model in the project to Deployment Space

### The following cell is a way to get the utility script required for this notebook. 
Since IBM CPD SaaS doesn't have a filesystem, this is the only reliable way to get scripts on the cloud environment. 
```
!rm -rf MLOps-CPD && git clone --quiet -b master https://github.com/IBM/MLOps-CPD.git
```
⚠️ Run the following cells only if you are executing on IBM CPD SaaS.

In [1]:
#!rm -rf MLOps-CPD && git clone --quiet -b master https://github.com/IBM/MLOps-CPD.git

In [ ]:
#!mv MLOps-CPD MLOps_CPD

## Imports

In [5]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_machine_learning import APIClient
from ibm_aigov_facts_client import AIGovFactsClient
from ibm_watson_studio_pipelines import WSPipelines
from botocore.client import Config
import ibm_boto3
import pandas as pd
import json
import os
import requests

from os.path import exists
if exists("utils/fs_utils.py") and exists("utils/catalog_utils.py"):
    from utils import fs_utils,catalog_utils
else:
    # If utils/fs_utils.py and utils/catalog_utils.py exist we assume that you are running on CPD SaaS
    # and will therefore import scripts from the freshly cloned repository
    from MLOps_CPD.utils import fs_utils, catalog_utils

## Pipeline Params

In [ ]:
CLOUD_API_KEY = os.getenv("cloud_api_key")
MODEL_NAME = os.getenv("model_name")
DEPLOYMENT_NAME = os.getenv("deployment_name")
space_id = os.getenv("space_id")
model_id = os.getenv('model_id')
project_id = os.environ['PROJECT_ID']

## Succeeding cell contains the credentials for MLOps COS
```
## PROJECT COS 
AUTH_ENDPOINT = "https://iam.cloud.ibm.com/oidc/token"
ENDPOINT_URL = "https://s3.private.us.cloud-object-storage.appdomain.cloud"
API_KEY_COS = "xxx"
BUCKET_PROJECT_COS = "mlops-donotdelete-pr-qxxcecxi1dtw94"

##MLOPS COS
ENDPOINT_URL_MLOPS = "https://s3.jp-tok.cloud-object-storage.appdomain.cloud"
API_KEY_MLOPS = "xxx"
CRN_MLOPS = "xxx"
BUCKET_MLOPS  = "mlops-asset"

##CATALOG
CATALOG_NAME = "MLOps-ns"
```

In [ ]:
# The code was removed by Watson Studio for sharing.

## Instantiate WML Client

In [ ]:
WML_CREDENTIALS = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey": CLOUD_API_KEY
            }

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

'1.0.257'

## Utility Functions

In [9]:
def get_model_from_registry(catalog_name):
    ctutils = catalog_utils.CatalogUtils(
    service_url="https://api.dataplatform.cloud.ibm.com",
    api_key=CLOUD_API_KEY,
    project_id=project_id,
    auth_url="https://iam.cloud.ibm.com/identity/token")

    catalog_id = ctutils.get_catalog_id_map()[catalog_name]
    model_id = ctutils.get_model_from_registry(MODEL_NAME)
    print(model_id)
    
    wml_client.set.default_project(project_id)
    model_pipeline = wml_client.repository.load(model_id)
    
    return model_id,catalog_id,model_pipeline


def read_data_from_mlops_cos(key,json=False):
    def __iter__(self): return 0
    MLOPS_DATA_STORE_client = ibm_boto3.client(
        service_name='s3',
        ibm_api_key_id=API_KEY_MLOPS,
        ibm_service_instance_id=CRN_MLOPS,
        ibm_auth_endpoint=AUTH_ENDPOINT,
        config=Config(signature_version='oauth'),
        endpoint_url=ENDPOINT_URL_MLOPS)

    body = MLOPS_DATA_STORE_client.get_object(Bucket=BUCKET_MLOPS, Key=key)['Body']
    # add missing __iter__ method, so pandas accepts body as file-like object
    if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
    if json:
        gcf_df = body
    else:
        gcf_df = pd.read_csv(body)
    return gcf_df

def promote_and_deploy_model(experiment_name):
    facts_client = AIGovFactsClient(api_key=CLOUD_API_KEY, experiment_name=experiment_name, container_type="project", container_id=project_id, set_as_current_experiment=True)
    fsutils = fs_utils.FSUtils(wml_client=wml_client,catalog_id=catalog_id,project_id=project_id,bss_account_id='27ff418fedd6aedffb8dc6ae4164a1d2',space_id=space_id,
                           facts_client=facts_client)
    result = fsutils.promote_model(model_uid=model_id,model_name=MODEL_NAME)
    dep_model_uid = result['metadata']['asset_id']
    deployment_uid = fsutils.deploy_model(space_id=space_id,deployment_name=DEPLOYMENT_NAME,model_uid=dep_model_uid)
    return deployment_uid,dep_model_uid


def test_deployment(CLOUD_API_KEY,deploymet_uid):
    # NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account.
    token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey":CLOUD_API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})

    mltoken = token_response.json()["access_token"]

    header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

    # NOTE: manually define and pass the array(s) of values to be scored in the next line

    payload_scoring = {"input_data": [{"fields": fields, "values": values}]}

    response_scoring = requests.post(f'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/{deployment_uid}/predictions?version=2022-10-12', json=payload_scoring,
     headers={'Authorization': 'Bearer ' + mltoken})

    print(response_scoring.json())
    if response_scoring.json():
        return True

## Load the Trained Model from Model Registry

In [10]:
model_id,catalog_id,model_pipeline = get_model_from_registry(CATALOG_NAME)

None


No "artifact_uid" provided.


MissingValue: No "artifact_uid" provided.

## Promote the Model to deployment space and Deploy the Model

In [11]:
deployment_uid,dep_model_uid = promote_and_deploy_model('CreditRiskModel')

2022/11/07 11:18:24 INFO : Successfully set CreditRiskModel as current experiment
2022/11/07 11:18:24 INFO : Autolog enabled Successfully
Unsetting the project_id ...
defaultdict(<class 'list'>, {})


#######################################################################################

Synchronous deployment creation for uid: '8cd59bfb-76fa-4f79-bb10-b0d82076bd32' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='2ff2d30e-9299-4865-bfbc-9fa4f977037d'
------------------------------------------------------------------------------------------------




## Model Testing on the Serving Endpoint



### Load Sample Data 

In [13]:
payload_data = read_data_from_mlops_cos('test_tfr.csv')
payload_data = payload_data.drop('Risk',axis=1)
fields = payload_data.columns.tolist()
values = [payload_data.values.tolist()[0]]

payload_scoring = {"input_data": [{"fields": fields, "values": values}]}
json.dumps(payload_scoring)

'{"input_data": [{"fields": ["LoanDuration", "LoanAmount", "InstallmentPercent", "CurrentResidenceDuration", "Age", "ExistingCreditsCount", "Dependents", "OwnsProperty", "CheckingStatus", "ExistingSavings", "LoanPurpose", "Telephone", "OthersOnLoan", "Job", "EmploymentDuration", "CreditHistory", "Housing", "Sex"], "values": [[31, 1889, 3, 3, 32, 1, 1, "savings_insurance", "0_to_200", "100_to_500", "other", "none", "none", "skilled", "less_1", "credits_paid_to_date", "own", "female"]]}]}'

## Score the Endpoint

In [14]:
predictions = wml_client.deployments.score(deployment_uid, payload_scoring)
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [[0, [0.7398784195684511, 0.26012158043154887]]]}]}

### Test for Downstream Apps without using WML SDK.

In [16]:
deploy_done = test_deployment(CLOUD_API_KEY,deployment_uid)
deploy_done

{'predictions': [{'fields': ['prediction', 'probability'], 'values': [[0, [0.7398784195684511, 0.26012158043154887]]]}]}


True

## Save Params in WS Pipeline

In [17]:
deployment_done = {}
deployment_done['deployment_status'] = deploy_done
deployment_done['deployment_id'] = deployment_uid
deployment_done['model_id'] = dep_model_uid
deployment_done['space_id'] = space_id

In [18]:
pipelines_client = WSPipelines.from_apikey(apikey=CLOUD_API_KEY)
pipelines_client.store_results(deployment_done)

Running outside of Watson Studio Pipeline - storing results in the local filesystem for testing purposes...

  output paths:
    - "deployment_status": .ibm_watson_studio_pipelines/results/deployment_status
    - "deployment_id": .ibm_watson_studio_pipelines/results/deployment_id
    - "model_id": .ibm_watson_studio_pipelines/results/model_id
    - "space_id": .ibm_watson_studio_pipelines/results/space_id
